In [1]:
# 파이썬으로 개발

!pip install openai
!pip install gradio

In [1]:
import openai
import gradio as gr
import apikey

OPENAI_API_KEY = apikey.OPENAI_API_KEY
openai.api_key = OPENAI_API_KEY

In [2]:
from IPython.display import Markdown, HTML
import time

class Chat:
    def __init__(self, system_content='You are a helpful assistant.'):
        self.system_content = system_content
        self.init_messages()
        
    def init_messages(self):
        self.messages = []
        if self.system_content:
            self.add_system_content(self.system_content)
            
    def add_user_content(self, content):
        self.messages.append({"role": "user", "content": content})
    def add_assistant_content(self, content):
        self.messages.append({"role": "assistant", "content": content})
    def add_system_content(self, content):
        self.messages.append({"role": "system", "content": content})
        
    def _run_query(self):
        max_retries = 7
        wait_time = 1

        for i in range(max_retries):
            try:
                self.response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages,
                    temperature=1.0, # control the level of randomness or "creativity" in the generated text.
                    top_p=1.0 # control the level of randomness or "creativity" in the next word.
                )
                return
            except Exception as e:
                if i == max_retries - 1:
                    raise
                else:
                    print(f"Exception {e}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                    wait_time *= 2

        
    def query(self, q, print_response=True):
        self.add_user_content(q)
        self._run_query()
        self.add_assistant_content(self.response['choices'][0]['message']['content'])
        if print_response:
            display(Markdown(self.messages[-1]['content']))
            
    def print_messages(self):
        for d in self.messages:
            role = d['role']
            content = d['content']
            if role == 'assistant':
                display(Markdown(content))
            else:
                color = '#080' if role == 'system' else '#008'
                display(HTML(f"<span style='color: {color}'><b>{role}</b> : {content}</span>"))

In [3]:
chat = Chat()

In [4]:
def add_text(state, text):
    chat.query(text)
    result = chat.messages[-1]['content']
    state = state + [(text, result)]
    return state, state


with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter, or upload an image").style(container=False)

    txt.submit(add_text, [state, txt], [state, chatbot])
    txt.submit(lambda :"", None, txt)

demo.launch(share=True, inline=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://0925a41f0eefb7a9de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


어쩔티비를 영어로는 "What to Watch TV" 또는 "What Should I Watch TV"로 번역할 수 있습니다.

In [5]:
# demo.close()

Closing server running on port: 7860
